# WAS6.11 - Diarrhoea cases data download+cleaning
Github issue: https://github.com/dataforgoodfr/batch8_actioncontrelafaim/issues/2

/!\ Some strata could be kept (see below)

## Settings

In [1]:
from pathlib import Path

NormalizedPath = lambda x: Path(os.path.normpath(os.path.normcase(x)))
import os

# conf
import sys
from path_manipulation import get_to_root

sys.path[0] = get_to_root(3,sys.path[0])
from config.config import config
sources = config.sources
meta = config.meta

In [2]:
# Target ressource
output_path = NormalizedPath(
    sources.was611_diarrhoea_cases_output_path
)
dataset_name = sources.was611_diarrhoea_dataset_name
ressource_index = sources.was611_diarrhoea_ressource_index
output_data_folder = NormalizedPath(sources.was611_diarrhoea_output_folder_path)
countries_ISO_3166_1_csv_path = NormalizedPath(meta.countries_ISO_3166_1_csv_path)

In [3]:
output_data_folder, countries_ISO_3166_1_csv_path

(WindowsPath('c:/users/charl/repositories/batch8_actioncontrelafaim/data/sources'),
 WindowsPath('c:/users/charl/repositories/batch8_actioncontrelafaim/data/meta/countries - iso 3166-1.csv'))

## Download

In [4]:
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset

In [5]:
try:
    Configuration.create(hdx_site="prod", user_agent="D4G_ACF", hdx_read_only=True)
except:
    pass
dataset = Dataset.read_from_hdx(dataset_name)
resources = dataset.get_resources()

In [6]:
# TODO / warning : the data should be erased first to have a real update ?
url, path = resources[ressource_index].download(folder=output_data_folder)
print("Resource URL %s downloaded to %s" % (url, path))

Resource URL https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/UNICEF,MNCH,1.0/.MNCH_DIARCARE.......?format=csv downloaded to c:\users\charl\repositories\batch8_actioncontrelafaim\data\sources\Careseeking for diarrhoea - percentage of children (under age 5) with diarrhoea for whom advice or treatment was sought from a health facility or provider (CSV).CSV


## Clean

In [7]:
# # Audit
# import pandas as pd
# df = pd.read_csv(path)
# print(df.columns, df.shape, df.notna().sum(axis=0))
# df.head()

In [8]:
import pandas as pd

df = pd.read_csv(path)
print(df.shape)
df = df[df.columns[df.notna().sum(axis=0) > 0]]
print(df.shape)
df = df[
    (df["Mother's Education Level"] == "Total")
    & (df["Sex"] == "Total")
    & (df["Wealth Quintile"] == "Total")
    & (df["Residence"] == "Total")
    & (df["Age at Birth"] == "Total")
    & (df["Current age"] == "Under 5 years old")
]

df = df.drop(
    columns=[
        "Mother's Education Level",
        "Sex",
        "Wealth Quintile",
        "Residence",
        "Age at Birth",
        "Current age",
    ]
)
print(df.shape)
assert df["Unit multiplier"].drop_duplicates().shape == (1,)
assert df["Unit of measure"].drop_duplicates().shape == (1,)
assert df["Observation confidentaility"].drop_duplicates().shape == (
    1,
)  # Reanalyzed ===> larger region e.g. Oceania
assert df["Observation Status"].drop_duplicates().shape == (
    2,
)  # Reanalyzed ===> larger region e.g. Oceania
assert df["SUB_SECTOR"].drop_duplicates().shape == (1,)
assert df[df["UNWGTD_SAMPL_SIZE"].notna()].sort_values("OBS_VALUE")[
    "Geographic area"
].shape < (
    35,
)  # highly missing information
assert df["OBS_FOOTNOTE"].notna().sum() == 0
df = df.drop(
    columns=[
        "Unit multiplier",
        "Unit of measure",
        "SUB_SECTOR",
        "Observation Status",
        "Observation confidentaility",
        "SUB_SECTOR",
        "UNWGTD_SAMPL_SIZE",
        "OBS_FOOTNOTE",
    ],
)
print(df.shape)
df = df.rename(
    columns={
        "Geographic area": "country_name",
        "Indicator": "indicator",
        "Data Source": "data_source",
        "TIME_PERIOD": "time_period",
        "OBS_VALUE": "indicator_value",
    }
)
df.head(2)

(4213, 25)
(4213, 18)
(374, 12)
(374, 5)


,country_name,indicator,data_source,time_period,indicator_value
4,Afghanistan,Careseeking for diarrhoea - percentage of chil...,Multiple Indicator Cluster Survey 2010-2011,2011,18.1
17,Afghanistan,Careseeking for diarrhoea - percentage of chil...,Demographic and Health Survey 2015,2015,54.2


## Complete with country codes

In [9]:
countries_iso = pd.read_csv(countries_ISO_3166_1_csv_path)
countries_iso = countries_iso[["name", "alpha3"]]
countries_iso.columns = ["country_name", "country_iso_alpha3"]
df = df.merge(
    countries_iso, left_on="country_name", right_on="country_name", how="inner"
)  # ignoring region with left join
print(df.shape)
df.head(2)

(331, 6)


,country_name,indicator,data_source,time_period,indicator_value,country_iso_alpha3
0,Afghanistan,Careseeking for diarrhoea - percentage of chil...,Multiple Indicator Cluster Survey 2010-2011,2011,18.1,afg
1,Afghanistan,Careseeking for diarrhoea - percentage of chil...,Demographic and Health Survey 2015,2015,54.2,afg


# Save

In [10]:
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, sep=";")

In [11]:
countries_iso

,country_name,country_iso_alpha3
0,Afghanistan,afg
1,Albania,alb
2,Algeria,dza
3,Andorra,and
4,Angola,ago
...,...,...
188,Venezuela (Bolivarian Republic of),ven
189,Viet Nam,vnm
190,Yemen,yem
191,Zambia,zmb
